In [13]:
import pandas as pd
import json

In [14]:
with open("../data/0_raw/casino_annotated.json") as f:
    raw = json.load(f)

In [15]:
raw = pd.DataFrame(raw)

In [16]:
raw["extend"] = raw.annotations.apply(lambda ls: (range(len(ls)), [dialogue[0] for dialogue in ls], [dialogue[1] for dialogue in ls]))

In [17]:
data = pd.DataFrame(columns= ["dialogue_id", "dialogue_turn", "text", "annotation"])

In [18]:
for _, df in raw.iterrows():
    data = data.append(pd.DataFrame({"dialogue_id": df.dialogue_id, 
                        "dialogue_turn": df.extend[0],
                        "text": df.extend[1],
                        "annotation": df.extend[2]}), ignore_index=True)
data

,dialogue_id,dialogue_turn,text,annotation
0,215,0,Hello! It will be so nice to go camping and s...,Small-talk
1,215,1,Yes it is. The scenery is beautiful and it is ...,Small-talk
2,215,2,I am working hard around the camp and getting ...,Small-talk
3,215,3,You came here to work? I’ve been taking it easy.,Small-talk
4,215,4,"LOL, I mean setting up the tent and clearing t...",Small-talk
...,...,...,...,...
4610,14,5,That sounds reasonable. I don't particularly n...,"No-Need,Self-Need,Showing-Empathy"
4611,14,6,I hear that. What do you think of if I take 3 ...,Non-strategic
4612,14,7,That sounds like a fair compromise. I can get ...,"Vouch-Fair,No-Need,Small-talk"
4613,14,8,That sounds fair to me! I don't know the first...,"Self-Need,Vouch-Fair,No-Need"


In [19]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb_result = mlb.fit_transform([str(data.loc[i,'annotation']).split(',') for i in range(len(data))])
data["multi_labels"] = mlb_result.tolist()
data = pd.concat([data, pd.DataFrame(mlb_result,columns=list(mlb.classes_))],axis=1)

In [20]:
data

,dialogue_id,dialogue_turn,text,annotation,multi_labels,,Elicit-Pref,No-Need,Non-strategic,Other-Need,Promote-coordination,Self-Need,Showing-Empathy,Small-talk,UV-Part,Vouch-Fair
0,215,0,Hello! It will be so nice to go camping and s...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,0,1,0,0
1,215,1,Yes it is. The scenery is beautiful and it is ...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,0,1,0,0
2,215,2,I am working hard around the camp and getting ...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,0,1,0,0
3,215,3,You came here to work? I’ve been taking it easy.,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,0,1,0,0
4,215,4,"LOL, I mean setting up the tent and clearing t...",Small-talk,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,14,5,That sounds reasonable. I don't particularly n...,"No-Need,Self-Need,Showing-Empathy","[0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0]",0,0,1,0,0,0,1,1,0,0,0
4611,14,6,I hear that. What do you think of if I take 3 ...,Non-strategic,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0,0,0,1,0,0,0,0,0,0,0
4612,14,7,That sounds like a fair compromise. I can get ...,"Vouch-Fair,No-Need,Small-talk","[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1]",0,0,1,0,0,0,0,0,1,0,1
4613,14,8,That sounds fair to me! I don't know the first...,"Self-Need,Vouch-Fair,No-Need","[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]",0,0,1,0,0,0,1,0,0,0,1


In [21]:
data.loc[data[""] == 1]

,dialogue_id,dialogue_turn,text,annotation,multi_labels,,Elicit-Pref,No-Need,Non-strategic,Other-Need,Promote-coordination,Self-Need,Showing-Empathy,Small-talk,UV-Part,Vouch-Fair
109,19,11,Yes. Water is the most important thing in life...,"Small-talk,Self-Need,,Vouch-Fair","[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1]",1,0,0,0,0,0,1,0,1,0,1


In [22]:
data.drop("",inplace=True, axis =1)
data.multi_labels = data.multi_labels.apply(lambda l: l[1:])
data

,dialogue_id,dialogue_turn,text,annotation,multi_labels,Elicit-Pref,No-Need,Non-strategic,Other-Need,Promote-coordination,Self-Need,Showing-Empathy,Small-talk,UV-Part,Vouch-Fair
0,215,0,Hello! It will be so nice to go camping and s...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
1,215,1,Yes it is. The scenery is beautiful and it is ...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
2,215,2,I am working hard around the camp and getting ...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
3,215,3,You came here to work? I’ve been taking it easy.,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
4,215,4,"LOL, I mean setting up the tent and clearing t...",Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,14,5,That sounds reasonable. I don't particularly n...,"No-Need,Self-Need,Showing-Empathy","[0, 1, 0, 0, 0, 1, 1, 0, 0, 0]",0,1,0,0,0,1,1,0,0,0
4611,14,6,I hear that. What do you think of if I take 3 ...,Non-strategic,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0,0,1,0,0,0,0,0,0,0
4612,14,7,That sounds like a fair compromise. I can get ...,"Vouch-Fair,No-Need,Small-talk","[0, 1, 0, 0, 0, 0, 0, 1, 0, 1]",0,1,0,0,0,0,0,1,0,1
4613,14,8,That sounds fair to me! I don't know the first...,"Self-Need,Vouch-Fair,No-Need","[0, 1, 0, 0, 0, 1, 0, 0, 0, 1]",0,1,0,0,0,1,0,0,0,1


In [34]:
class_dict = {k: v for k,v in enumerate(data.columns[-10:])}

In [36]:
class_dict

{0: 'Elicit-Pref',
 1: 'No-Need',
 2: 'Non-strategic',
 3: 'Other-Need',
 4: 'Promote-coordination',
 5: 'Self-Need',
 6: 'Showing-Empathy',
 7: 'Small-talk',
 8: 'UV-Part',
 9: 'Vouch-Fair'}

In [37]:
data[class_dict.values()].sum()

Elicit-Pref              377
No-Need                  196
Non-strategic           1455
Other-Need               409
Promote-coordination     579
Self-Need                964
Showing-Empathy          254
Small-talk              1054
UV-Part                  131
Vouch-Fair               439
dtype: int64

In [38]:
data[data["Other-Need"]==1].text.tolist()

["That sounds like so much fun!  I hope we get to hike some.  I have my four month old baby with us which is exciting.  I'm worried I didn't bring enough water for his bottles, though.",
 'That sucks, you need to be hydrated. What supplies do you require? My family loves to grill, so we do require food.',
 'Hello. I would consider giving you 2 waters for 3 firewood and 2 food. I have a large family so the additional firewood will help with warmth and cooking food. 🙂',
 "Yes it is very much needed. My family doesn't have a lot of extra bedding so having the warmth of a fire will help make sure we don't get cold at night 🙂",
 'Well, I have two teenage boys who will be with me.  In normal circumstances they go through a lot of food.  While camping, hiking and kayaking I know we will need plenty to keep their energy level high to do everything.🙂',
 'I have a very fast metabolism and a dog to take care of. Thanks for the firewood.',
 "I'm so sorry, we're getting very close to the end and wi

In [39]:
data

,dialogue_id,dialogue_turn,text,annotation,multi_labels,Elicit-Pref,No-Need,Non-strategic,Other-Need,Promote-coordination,Self-Need,Showing-Empathy,Small-talk,UV-Part,Vouch-Fair
0,215,0,Hello! It will be so nice to go camping and s...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
1,215,1,Yes it is. The scenery is beautiful and it is ...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
2,215,2,I am working hard around the camp and getting ...,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
3,215,3,You came here to work? I’ve been taking it easy.,Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
4,215,4,"LOL, I mean setting up the tent and clearing t...",Small-talk,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,14,5,That sounds reasonable. I don't particularly n...,"No-Need,Self-Need,Showing-Empathy","[0, 1, 0, 0, 0, 1, 1, 0, 0, 0]",0,1,0,0,0,1,1,0,0,0
4611,14,6,I hear that. What do you think of if I take 3 ...,Non-strategic,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",0,0,1,0,0,0,0,0,0,0
4612,14,7,That sounds like a fair compromise. I can get ...,"Vouch-Fair,No-Need,Small-talk","[0, 1, 0, 0, 0, 0, 0, 1, 0, 1]",0,1,0,0,0,0,0,1,0,1
4613,14,8,That sounds fair to me! I don't know the first...,"Self-Need,Vouch-Fair,No-Need","[0, 1, 0, 0, 0, 1, 0, 0, 0, 1]",0,1,0,0,0,1,0,0,0,1


In [40]:
data.to_csv("../data/1_interim/annotated_dialogues.csv")